# Prompt Engineering

### Author: Sharon Zhou <<sharonz@cs.stanford.edu>>. CS Dept Faculty, Stanford University.

### This notebook lives [here](https://colab.research.google.com/drive/1P4W-V_JDUs1bOm6c5Bfn6aG5HtDx4QZm).

#### In this notebook, you'll get change and play with the data that is inputed to LLMs in a programmatic way.

#### _Goal: See how even small amounts of data can make an LLM much more useful!_


In [29]:
# @title Setup: Authenticate with Google & install the open-source [PowerML library](https://pypi.org/project/powerml-app/0.0.35/) to use LLMs easily
%%capture

from google.colab import auth
import requests

def authenticate_powerml():
  auth.authenticate_user()
  gcloud_token = !gcloud auth print-access-token
  powerml_token_response = requests.get('https://api.staging.powerml.co:5002/auth/verify_gcloud_token?token=' + gcloud_token[0])
  return powerml_token_response.json()['token']

token = authenticate_powerml()
!pip install powerml-app==0.0.41

config = {"powerml": { "key": token, "url": "https://api.staging.powerml.co" }}

# print('Ready to play with LLMs!')

### Scenario

You have a lot of email you have to write for the different extracurriculars: movie, running, and math clubs. So, you want to use an LLM to help you write them. 


### Exercise 1

You have the following code written, so your LLM can write emails for all your clubs. But it writes a random email. You have no control over what the subject of them will be.

**Your task: Adjust the context to enable you to specify the subject of the email.**

_Goal: Learn how to build reusable contexts/prompts!_


In [ ]:
from powerml import LLM, ContextTemplate

# Build a template for giving context to your LLM; TODO: change the context to take in any subject
context = "Write an email for the {{club_name}} club." # double curly braces indicate a variable

# Specify the variables in the context as arguments
# Helpful reference when you have more arguments **hint**
args=["club_name"]

# Instantiate the context template
template = ContextTemplate(context, args)

# Instantiate your LLM
llm = LLM(config)

# Fit your LLM to the context
llm.fit(template)

my_club_names = ['movie', 'running', 'math']
for my_club_name in my_club_names:
  # Run your LLM, looping through your different clubs
  output = llm.predict(club_name=my_club_name)
  print(output)
  print("--------------")

In [ ]:
# SOLUTION
from powerml import LLM, ContextTemplate

# Build a template for giving context to your LLM
context = "Write an email about {{subject}} for the {{club_name}} club." # double curly braces indicate a variable

# Specify the variables in the context as arguments
args=["subject", "club_name"]

# Instantiate the context template
template = ContextTemplate(context, args)

# Instantiate your LLM
llm = LLM(config)

# Fit your LLM to the context
llm.fit(template)

my_club_names_subjects = {
    'movie': ['recruiting new members', 'the new harry potter fan fiction indie movie'],
    'running': ['the team winning the meet', 'practice being still on despite the blizzard'],
    'math': ['competition practice', 'electing a new president for next semester']
}

for my_club_name, subjects in my_club_names_subjects.items():
  # Run your LLM, looping through your different clubs and subjects for them
  for subject in subjects:
    print(f'Email for {my_club_name} club about {subject}')
    output = llm.predict(subject=subject, club_name=my_club_name)
    print(output)
    print("\n--------------")

### Exercise 2
The emails you're seeing don't sound like you. You sound better, funnier, or at least more like _you_! Good thing you have a few examples of emails you've written before for each club.

**Your task: Given your past emails (provided for you), build a context template to include your past emails and fit a new LLM. Run and compare results.**

_Goal: Observe the impact of more data._

How does it do?

In [44]:
# @markdown #### Setup: Fetch email files

!wget -q -O "movie_club.txt" "https://drive.google.com/uc?export=download&id=1dWKvcCIp-XwNBZ6heb5k2Y1tbdPl3WTE"
!wget -q -O "running_club.txt" "https://drive.google.com/uc?export=download&id=1SxeE-0XP4ggjUNYwMD3VQlm_idCHoG9G"
!wget -q -O "math_club.txt" "https://drive.google.com/uc?export=download&id=12z3EuOauuYc4Y9tdDGENs-_2wmvyErmv"

print('Got all the emails!')

Got all the emails!


In [50]:
with open('movie_club.txt', 'r') as f:
  past_movie_club_emails = f.read()
with open('running_club.txt', 'r') as f:
  past_running_club_emails = f.read()
with open('math_club.txt', 'r') as f:
  past_math_club_emails = f.read()

# TODO (same code as above, without your changes for subject in Exercise 1)
context = "Write an email for the {{club_name}} club."
args=["club_name"]
template = ContextTemplate(context, args)

llm = LLM(config)
llm.fit(template)

my_club_names = ['movie', 'running', 'math']
for my_club_name in my_club_names:
  output = llm.predict(club_name=my_club_name)
  print(output)
  print("--------------")

In [ ]:
# SOLUTION
with open('movie_club.txt', 'r') as f:
  past_movie_club_emails = f.read()
with open('running_club.txt', 'r') as f:
  past_running_club_emails = f.read()
with open('math_club.txt', 'r') as f:
  past_math_club_emails = f.read()

context = "{{past_emails}}\n\nBased on the tone of past emails, write an email about {{subject}} for the {{club_name}} club." # double curly braces indicate a variable
args=["past_emails", "subject", "club_name"]
template = ContextTemplate(context, args)

llm = LLM(config)
llm.fit(template)

my_clubs_info = [
    {
      'name': 'movie',
      'subjects': ['recruiting new members', 'the new harry potter fan fiction indie movie'],
      'past_emails': past_movie_club_emails
    },
    {
      'name': 'running',
      'subjects': ['the team winning the meet', 'practice being still on despite the blizzard'],
      'past_emails': past_running_club_emails
    },
    {
      'name': 'math',
      'subjects': ['competition practice', 'electing a new president for next semester'],
      'past_emails': past_math_club_emails
    }
]

for club in my_clubs_info:
  my_club_name = club['name']
  subjects = club['subjects']
  past_emails = club['past_emails']

  # Run your LLM, looping through your different clubs and subjects for them
  for subject in subjects:
    print(f'Email for {my_club_name} club about {subject}')
    output = llm.predict(past_emails=past_emails, subject=subject, club_name=my_club_name)
    print(output)
    print("\n--------------")

### Exercise 3

So you think you're funny and you prize how funny you are. But what if you can get the LLM to make you funnier?

**Your task: Engineer the prompt such that you can sound funnier than you.**

_Goal: Play with the model, and see where it's easy to manipulate and where it isn't. And that it is not always intuitive._


In [ ]:
# TODO (same code as Exercise 2 to get you started, without your changes)

with open('movie_club.txt', 'r') as f:
  past_movie_club_emails = f.read()
with open('running_club.txt', 'r') as f:
  past_running_club_emails = f.read()
with open('math_club.txt', 'r') as f:
  past_math_club_emails = f.read()

context = "Write an email for the {{club_name}} club."
args=["club_name"]
template = ContextTemplate(context, args)

llm = LLM(config)
llm.fit(template)

my_club_names = ['movie', 'running', 'math']
for my_club_name in my_club_names:
  output = llm.predict(club_name=my_club_name)
  print(output)
  print("--------------")

There are many right answers, so don't worry if this doesn't match your completely! Humor is subjective, after all :)

In [ ]:
# SOLUTION

with open('movie_club.txt', 'r') as f:
  past_movie_club_emails = f.read()
with open('running_club.txt', 'r') as f:
  past_running_club_emails = f.read()
with open('math_club.txt', 'r') as f:
  past_math_club_emails = f.read()

context = "{{past_emails}}\n\nBased on the tone of past emails, write a very funny email about {{subject}} for the {{club_name}} club. Be sure to include a joke or witty pun." # double curly braces indicate a variable
args=["past_emails", "subject", "club_name"]
template = ContextTemplate(context, args)

llm = LLM(config)
llm.fit(template)

my_clubs_info = [
    {
      'name': 'movie',
      'subjects': ['recruiting new members', 'the new harry potter fan fiction indie movie'],
      'past_emails': past_movie_club_emails
    },
    {
      'name': 'running',
      'subjects': ['the team winning the meet', 'practice being still on despite the blizzard'],
      'past_emails': past_running_club_emails
    },
    {
      'name': 'math',
      'subjects': ['competition practice', 'electing a new president for next semester'],
      'past_emails': past_math_club_emails
    }
]

for club in my_clubs_info:
  my_club_name = club['name']
  subjects = club['subjects']
  past_emails = club['past_emails']

  # Run your LLM, looping through your different clubs and subjects for them
  for subject in subjects:
    print(f'Email for {my_club_name} club about {subject}')
    output = llm.predict(past_emails=past_emails, subject=subject, club_name=my_club_name)
    print(output)
    print("\n--------------")

#### Great work! Hope you saw how small programmatic changes to the context for the LLM made sometimes drastic changes and improvements to how the LLM behaved. Now off to do more things with LLMs!